## C S 329E HW 7

# Naive Bayes 

## Jazmin Reyna

For this week's homework we are going explore one new classification technique:

  - Naive Bayes

We are reusing the version of the Melbourne housing data set from HW6, to predict the housing type as one of three possible categories:

  - 'h' house
  - 'u' duplex
  - 't' townhouse

In addition to building our own Naive Bayes classifier, we are going to compare the performace of our classifier to the [Gaussian Naive Bayes Classifier](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes) available in the scikit-learn library. 

In [1]:
# These are the libraries you will use for this assignment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import calendar
from sklearn.naive_bayes import GaussianNB # The only thing in scikit-learn you can use this assignment

# Starting off loading a training set and setting a variable for the target column, "Type"
df_melb = pd.read_csv('https://gist.githubusercontent.com/yanyanzheng96/81b236aecee57f6cf65e60afd865d2bb/raw/56ddb53aa90c26ab1bdbfd0b8d8229c8d08ce45a/melb_data_train.csv')
target_col = 'Type'

## Q1 - Fix a column of data to be numeric
If we inspect our dataframe, `df_melb` using the `dtypes` method, we see that the column "Date" is an object.  However, we think this column might contain useful information so we want to convert it to [seconds since epoch](https://en.wikipedia.org/wiki/Unix_time). Use only the exiting imported libraries to create a new column "unixtime". Be careful, the date strings in the file might have some non-uniform formating that you have to fix first.  Print out the min and max epoch time to check your work.  Drop the original "Date" column. 

THESE ARE THE EXACT SAME INSTRUCTIONS FROM HW6! Please take this opportunity to reuse your code (if you got it right last time). 

In [2]:
# normalize date accepts the date string as shown in the df_melb 'Date' column,
# and returns a data in a standarized format
def standardize_date(d):
    string = d.split('/')
    for i in range(0,3):
        if i != 2:
            if len(string[i]) < 2:
                string[i] = '0' + string[i]
        else:
            if len(string[i]) < 4:
                string[i] = '20' + string[i]
    space = ''
    new_date = space + string[0] + '/' + string[1] + '/' + string[2]
    return time.strptime(new_date, '%d/%m/%Y')


In [3]:
df_melb['Date'] = df_melb['Date'].apply( standardize_date )
df_melb['unixtime'] = df_melb['Date'].apply( lambda x : calendar.timegm(x))
df_melb = df_melb.drop(columns="Date")


## Q2 Calculating the prior probabilities
Calculate the prior probabilities for each possible "Type" in `df_melb` and populate a dictionary, `dict_priors`, where the key is the possible "Type" values and the value is the prior probabilities. Show the dictionary. Do not hardcode the possible values of "Type".  Don't forget about [value counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html). 

In [4]:
dict_priors = {}

types = (df_melb.Type.value_counts(normalize = True))
count = 0
for i in types.keys():
    dict_priors[i] = types[count]
    count += 1
    
dict_priors

{'h': 0.452, 'u': 0.418, 't': 0.13}

## Q3 Create a model for the distribution of all of the numeric attributes
For each class, and for each attribute calculate the sample mean and sample standard deviation.  You should store the model in a nested dictionary, `dict_nb_model`, such that `dict_nb_model['h']['Rooms']` is a tuple containing the mean and standard deviation for the target Type 'h' and the attribute 'Rooms'.  Show the model using the `display` function. You should ignore entries that are `NaN` in the mean and [standard deviation](https://pandas.pydata.org/docs/reference/api/pandas.Series.std.html) calculation. 

In [5]:
dict_nb_model = dict()

for target in dict_priors.keys():
    dict_nb_model[target] = {}
    df_type = df_melb[df_melb['Type'] == target]
    for col in df_type.columns:
        if col != 'Type':
            dict_nb_model[target][col] = (df_type[col].mean(skipna=True), df_type[col].std(skipna=True))

In [6]:
display(dict_nb_model)

{'h': {'Rooms': (3.269911504424779, 0.7258264201127756),
  'Price': (1189022.3451327435, 586296.5794417895),
  'Distance': (12.086725663716809, 7.397501132737295),
  'Postcode': (3103.8982300884954, 98.35750345419703),
  'Bathroom': (1.5619469026548674, 0.6720871086493075),
  'Car': (1.7777777777777777, 0.932759177140425),
  'Landsize': (932.9646017699115, 3830.7934157687164),
  'BuildingArea': (156.24339622641511, 54.62662837301434),
  'YearBuilt': (1954.900826446281, 32.461876347154686),
  'unixtime': (1485717578.761062, 13838562.050601462)},
 'u': {'Rooms': (2.0430622009569377, 0.5908453859944267),
  'Price': (634207.1770334928, 217947.32866736987),
  'Distance': (8.760287081339715, 5.609778714430755),
  'Postcode': (3120.4545454545455, 87.18475679946482),
  'Bathroom': (1.1818181818181819, 0.42228151548662185),
  'Car': (1.1483253588516746, 0.47231993860296956),
  'Landsize': (436.23444976076553, 1394.3403794653254),
  'BuildingArea': (83.85585585585585, 45.959438015166604),
  'Yea

## Q4 Write a function that calculates the probability of a Gaussian
Given the mean ($\mu$), standard deviation ($\sigma$), and a observed point, `x`, return the probability.  
Use the formula $p(x) = \frac{1}{\sigma \sqrt{2 \pi}} e^{-\frac{1}{2}(\frac{x-\mu}{\sigma})^2}$ ([wiki](https://en.wikipedia.org/wiki/Normal_distribution)).  You should use [numpy's exp](https://numpy.org/doc/stable/reference/generated/numpy.exp.html) function in your solution. 

In [7]:
def get_p( mu, sigma, x):
    d = (sigma*(np.sqrt(2*np.pi)))
    e = (-(1/2)*(((x - mu)/sigma)**2))
    return (1/d)*np.exp(e)

In [8]:
# Test it. You should get a value of 0.19333405840142462
p = get_p( 0, 2, 0.5)
p

0.19333405840142462

## Q5 Write the Naive Bayes classifier function
The Naive Bayes classifier function, `nb_class`, should take as a parameter the prior probability dictionary. `dict_priors`, the dictionary containing all of the gaussian distribution information for each attribue, `dict_nb_model`, and a single observation row (a series generated from iterrows) of the test dataframe. It should return a single target classification. For this problem, all of our attributes are numeric and modeled as Gaussians, so we don't worry about categorical data. Make sure to skip attributes that do not have a value in the observation.  Do not hardcode the possible classification types. 

In [9]:
def nb_class( dict_priors, dict_nb_model, observation):
    # your code here  
    probs = {}
    for i,j in dict_priors.items():
        p = j
        for k in observation.keys():
            if k != 'Type':
                if np.isnan(observation[k]) == False:
                    cond = get_p(dict_nb_model[i][k][0], dict_nb_model[i][k][1], observation[k])
                    p *= cond
        probs[i] = p
    return max(probs, key = probs.get)
        

## Q6 Calculate the accuracy using Naive Bayes classifier function on the test set
Load the test set from file, convert date to unix time and drop the date column, classify each row using your `nb_class`, and then show the accuracy on the test set. 

In [10]:
df_test = pd.read_csv('https://gist.githubusercontent.com/yanyanzheng96/c3d53303cebbd986b166591d19254bac/raw/94eb3b2d500d5f7bbc0441a8419cd855349d5d8e/melb_data_test.csv')
df_test['Date'] = df_test['Date'].apply( standardize_date )
df_test['unixtime'] = df_test['Date'].apply( lambda x : calendar.timegm(x))
df_test = df_test.drop(columns="Date")

In [11]:
predictions = []

for (indx,row) in df_test.iterrows():
    predictions.append(nb_class(dict_priors, dict_nb_model, row))

In [12]:
acc = np.sum(df_test['Type'] == predictions)/len(df_test['Type']) # your code here

In [13]:
# You should get and accuracy of 57.00%
print('Accuracy is {:.2f}%'.format(acc*100))

Accuracy is 57.00%


## Use scikit-learn to do the same thing!

Now we understand the inner workings of the Naive Bayes algorithm, let's compare our results to [scikit-learn's Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) implementation. Use the [GaussianNB](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes) to train using the `df_melb`dataframe and test using the `df_test` dataframe. Remember to split `df_melb` into a `df_X` with the numerical attributes, and a `s_y` with the target column. On the `df_melb` frame you will have to fill the empty attributes via imputation since the scikit-learn library can not handle missing values.  Use the same method you used in the last homework (filling the training data with the mean of the non-nan values). 

Answer the following in a markdown cell: do you think imputation hurt or helped the classifier?

In [14]:
# Imputation training
dict_imputation = dict()

for col in df_melb.columns:
    if col != 'Type':
        dict_imputation[col] = df_melb[col].mean()
        df_melb[col].fillna(value = df_melb[col].mean(), inplace=True)      

# Imputation - apply on the test data
for col in df_test.columns:
    if col != 'Type':
        df_test[col].fillna(value = dict_imputation[col], inplace=True)
        
# Seperate the attributes from the target_col
s_y = df_melb['Type']
df_X = df_melb.drop(columns = 'Type')

In [15]:
gnb = GaussianNB()

gnb.fit(df_X, s_y)

p_test = gnb.predict(df_test.drop(columns = 'Type'))

In [16]:
acc = np.sum(df_test['Type'] == p_test)/len(df_test['Type'])# your code here
print('Accuracy is {:.2f}%'.format(acc*100))

Accuracy is 37.00%


## I believe imputation hurt the classifier as the accuracy is now much lower